In [ ]:
import Graphics.Color.Demo
import Graphics.Color.Space
import Graphics.Image as I
:set -XDataKinds
:set -XTypeApplications
:set -XPartialTypeSignatures

In [ ]:
:! xdg-open https://hackage.haskell.org/package/Color-0.2.0/docs/Graphics-Color-Space.html#t:ColorSpace > /dev/null 2>&1
:! xdg-open https://github.com/lehins/Color/blob/6a3932da974132a5ec87f7f97237e9e659f53e48/Color/src/Graphics/Color/Space/Internal.hs#L66-L104 > /dev/null 2>&1

In [ ]:
display . I.scale Bilinear Edge (0.5, 0.5) =<< I.readImageRGB "files/LMS.png"
-- Source: http://jamie-wong.com/post/color/

In [ ]:
-- William David Wright
display . I.scale Bilinear Edge (0.5, 0.5) =<< I.readImageRGB "files/CIERGB.png"

In [ ]:
display . I.scale Bilinear Edge (0.5, 0.5) =<< I.readImageRGB "files/CIERGB-spectral-locus.png"

In [ ]:
display =<< I.readImageRGB "files/CIERGB-gamut.jpg"

In [ ]:
import Graphics.Color.Illuminant.CIE1931
import qualified Graphics.Color.Space.CIE1931.RGB as CIE
cieGamut = gamut :: Gamut CIE.CIERGB _ Double
cieGamut
:t cieGamut
:t gamut

In [ ]:
gamutWhitePoint cieGamut

In [ ]:
display . I.scale Bilinear Edge (0.5, 0.5) =<< I.readImageRGB "files/CIExy1931_AdobeRGB_vs_sRGB.png"

In [ ]:
import Graphics.Color.Space.RGB.AdobeRGB
print (gamut :: Gamut SRGB D65 Float)
print (gamut :: Gamut AdobeRGB D65 Float)

In [ ]:
print (colorTemperature :: CCT D65)
print (whitePoint :: WhitePoint D65 Float)

In [ ]:
:! xdg-open https://hackage.haskell.org/package/Color-0.2.0/docs/Graphics-Color-Space.html#t:Illuminant > /dev/null 2>&1

In [ ]:
sRGB = ColorRGB 0.1 0.9 0.2 :: Color (SRGB 'NonLinear) Float
sRGB
displayColor sRGB
:t ColorRGB

In [ ]:
convertColor sRGB :: Color (AdobeRGB 'NonLinear) Float
:t convertColor

In [ ]:
-- unwrap `convertColor`
adobeRGB = (ecctf :: Color (AdobeRGB 'Linear) Float -> Color (AdobeRGB 'NonLinear) Float) -- Encoding color component transfer function
         . (inpmApply inpm :: Color (XYZ D65) Float -> Color (AdobeRGB 'Linear) Float) -- Inverse normalized primary matrix
         . (npmApply npm :: Color (SRGB 'Linear) Float -> Color (XYZ D65) Float) -- Normalized primary matrix
         . (dcctf :: Color (SRGB 'NonLinear) Float -> Color (SRGB 'Linear) Float) -- Decoding color component transfer function
         $ sRGB
sRGB
adobeRGB

In [ ]:
convertColor sRGB :: Color (XYZ D65) Float
convertColor sRGB :: Color (CIExyY D65) Float

In [ ]:
makeImage (Sz2 100 1100) (\(_ :. j) -> PixelRGB 0 0 (fromIntegral ((255 * j) `div` 1100)) :: Pixel (SRGB 'NonLinear) Word8)
foldl1 leftToRight [makeImage (Sz 100) $ const (PixelRGB 0 0 (25 * i) :: Pixel (SRGB 'NonLinear) Word8 ) | i <- [0 :: Word8 .. 10]]

In [ ]:
makeImage (Sz2 100 1100) (\(_ :. j) -> PixelRGB 0 0 (fromIntegral ((255 * j) `div` 1100)) :: Pixel (SRGB 'Linear) Word8)
foldl1 leftToRight [makeImage (Sz 100) $ const (PixelRGB 0 0 (25 * i) :: Pixel (SRGB 'Linear) Word8) | i <- [0 :: Word8 .. 10]]

In [ ]:
:t npm 
print (inpm :: INPM SRGB Float)
print (npm :: NPM SRGB Float)
-- https://en.wikipedia.org/wiki/SRGB#Specification_of_the_transformation

In [ ]:
import qualified Graphics.Color.Space.RGB.Derived.SRGB as Derived
import qualified Graphics.Color.Illuminant.Wikipedia as W
print (inpm :: INPM (Derived.SRGB D65) Float)
print (npm :: NPM (Derived.SRGB D65) Float)

In [ ]:
:! xdg-open https://en.wikipedia.org/wiki/SRGB#Specification_of_the_transformation > /dev/null 2>&1

In [ ]:
sRGB
convertColor sRGB :: Color (Derived.SRGB D65 'NonLinear) Float